In [1]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import time
import re

In [213]:
df = pd.read_excel('C:/Develops/Project_multi_A3/scraping/news/NewsResult_20200901-20210430.xlsx', names=['identical', 'date', 'press', 'name', 'title', 'c1', 'c2', 'c3', 'a1', 'a2', 'a3', 'person', 'place', 'institute', 'keyword', 'topkeyword', 'body', 'url', 'tf'])

# 전처리
df = df[df.tf != '예외']
df = df[df.tf != '중복']
df = df[df.tf != '중복, 예외']
df = df[~df.title.str.contains('경향포토')]
df = df[~df.title.str.contains('인터랙티브')]
df = df[~df.place.str.contains('korea', na=False)]
df = df[~df.place.str.contains('la', na=False)]
df = df[~df.place.str.contains('LA', na=False)]
df = df.reset_index()
df = df.drop(columns=['index'], axis=1)

In [135]:
df.head(40)

,identical,date,press,name,title,c1,c2,c3,a1,a2,a3,person,place,institute,keyword,topkeyword,body,url,tf
0,1.100101e+06,20210430,경향신문,하경헌 기자 azimae@kyunghyang.com,"원태인, 평균자책 1위 ‘젊은 사자’의 포효",스포츠>야구,스포츠>농구_배구,스포츠>올림픽_아시안게임,NaN,NaN,NaN,"데이비드 뷰캐넌,구창모,미란다,원태인,호세,고영표,아리엘,앤드류,이정용,피렐라","수원,4경기,삼성라이온즈파크,0.5경기,경기,도쿄,대구,잠실,5경기","KIA,LG 상대,SSG,LG전,KT,우전,삼성,ㆍ삼성,삼진,NC,두산,SSG전,LG","원태,평균,자책,사자,포효,무실점,LG,상대,5경기,4승,공동,삼성,LG,0.5경기...","원태인,7이닝,lg,삼성,평균자책,땅볼,1위,무실점,이형종,6이닝,두산,삼자범퇴,이...",ㆍLG 상대 7이닝 5안타 무실점 \nㆍ5경기 만에 4승으로 ‘공동 1위’ \nㆍ삼...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
1,1.100101e+06,20210430,경향신문,박효순 기자 anytoc@kyunghyang.com,"“한의약 의료서비스의 건보 확대, 국민 건강 증진에 도움”",사회>의료_건강,사회>여성,사회>장애인,NaN,NaN,NaN,박성우,"수유,서울특별시,안면,서울시,강남구한의사회,서울시내","해외의료봉사단,한의원,경희대학교,서울시한의사회,경희대,경희보궁한의원,한의약,초음파장...","한의약,의료,서비스,건보,확대,증진,국민,건강,도움,회장,박성우,서울시한의,사회,월...","한의약,첩약,난임,한의원,박성우,한의대,치료비,시범사업,한의사,후유증,월경통,회원들...",ㆍ박성우 서울시한의사회 회장\n\n\n\n월경통 등 첩약 건보 시범사업\n추나요법처...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
2,1.100101e+06,20210430,경향신문,군산 | 이정호 기자 alpha@kyunghyang.com,불편한 여론도 잠재운 골프장의 ‘박찬호 효과’,스포츠>골프,스포츠>축구,스포츠>올림픽_아시안게임,NaN,NaN,NaN,"최호성,박찬호,마관우","영암,전남,슬럼프,군산,함정,군산CC오픈","코리안,한국여자프로골프(KLPGA,한국프로골프(,하나금융,KPGA,KLPGA,해피콜","불편,여론,골프장,박찬호,효과,독점,코리안,투어,관심,흥행,도움,긍정,분위기,2라운...","박찬호,코리안투어,대회장,코리안,코로나19,한국,2라운드,취재진,군산,29오버,kl...",ㆍ코리안투어 ‘관심 독점’ 우려에서 ‘흥행에 도움’ 긍정적 분위기로 \nㆍ2라운드 ...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
3,1.100101e+06,20210430,경향신문,조홍민 선임기자 dury129@kyunghyang.com,전창진 vs 김승기 ‘승부는 인연보다 뜨겁다’,스포츠>축구,스포츠>야구,스포츠>농구_배구,사고>스포츠사고,NaN,NaN,"김,송교창,전,이정현,오세근,전창진,설린저,라건아,제러드,김승기","안양,전주,서울,원주,강남구,전북,경기","동부,한솥,KBL,KGC,KCC·KGC,전주 KCC,KCC","전창진,vs,김승기,승부,인연,KGC,프로,농구,KCC,KGC,감독,선후배,사제,인...","설린저,챔피언결정전,전창진,김승기,kgc,결정전,kcc,1차,전주체육관,미디어데이,...",ㆍ프로농구 KCC KGC의 두 감독 \nㆍ선후배 사제 관계 등 질긴 인연 \nㆍ챔피...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
4,1.100101e+06,20210430,경향신문,윤은용 기자 plaimstone@kyunghyang.com,‘후배 주취폭행’ 현대모비스 기승호 ‘제명’,스포츠>농구_배구,스포츠>축구>한국프로축구,스포츠>축구>해외축구,범죄>범죄일반>폭행,사고>스포츠사고,NaN,"장재석,기승호","울산,풍덕천,안양,서울,논현동,용인시","재정위원회,KGC인삼공사,KBL,용인서부경찰서,현대모비스,재정위,ㆍKBL","후배,주취,폭행,제명,현대모비스,기승호,ㆍKBL,재정위,징계,최고,수위,음주,후배,...","기승호,현대모비스,장재석,코로나19,재정위원회,안양,플레이오프,a씨,위원회,선수단,...","ㆍKBL 재정위, 최고 수위 징계\n\n\n\n음주 뒤 후배를 폭행한 프로농구 울산...",http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
5,1.100101e+06,20210430,경향신문,김유익 재중문화교류활동가,[김유익의 광저우 책갈피]탈출구 없는 초경쟁사회가 된 중국 사라진 공동체 멘토를 불러내다,국제>중국,NaN,NaN,NaN,NaN,NaN,"다케우치,루쉰","아시아,중,중국,홍콩,센트럴,톈안먼,옥스퍼드,서구,샹뱌오,한국,한,동아시아","요시미,일본,중국,쉬즐유엔,현대,조선,대국,베이징대,창비","탈출구,초경쟁사회,중국,공동체,멘토,경계인,중국,민주화,샹뱌오,방법,샹뱌오,대화,블...","중국,홍콩,동아시아,샹뱌오,대담집,경계인,공동체,인류학자,김유익,현대,탈출구,루쉰,...",ㆍ경계인이 본 중국의 민주화\n\n\n\n샹뱌오\n<방법으로서의 자기 - 샹뱌오와의...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
6,1.100101e+06,20210430,경향신문,NaN,[새책]바늘과 가죽의 시 外,문화>출판,문화>미술_건축,문화>전시_공연,사고>산업사고>원자력사고,NaN,NaN,"나희덕,구병모,류이치,바바라,아녜스","미국,한국,뉴베리,체르노보,체르노빌","노벨상,알리나,현대,마음산책","바늘,가죽,바늘,가죽,소설가,구병모,신작,요정,동화,구두,장이,기원,인간화,요정,이...","1만,릴리,체르노보,바바라,두냐,켈러,체르노빌,브론스키,알리나,에코,미국,한국,소설...","바늘과 가죽의 시\n\n소설가 구병모의 신작. 동화 <구두장이 요정>에서 기원한, ...",http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
7,1.100101e+06,20210430,경향신문,김지혜 기자 kimg@kyunghyang.com,"[책과 삶]침묵하지만, 들리는 위안부의 고통",국제>일본,문화>출판,문화>학술_문화재,NaN,NaN,NaN,김숨,재구,NaN,"침묵,위안부,고통,김숨,문학실험실,176쪽,1만,증언,기록,장편,소설,위안부,피해자...","위안부,황수남,피해자,성윤주,김숨,고통들,단편,재구성,단어들,히스테릭,단상들,정신병...",듣기 시간\n김숨 지음\n문학실험실 | 176쪽 | 1만6000원\n\n말이 되지 ...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
8,1.100101e+06,20210430,경향신문,심윤지 기자 sharpsim@kyunghyang.com,"[이미지로 여는 책]식물학자가 그린 과학 일러스트, 식물들의 경이로운 투쟁사 담다",문화>출판,문화>미술_건축,사회>환경,NaN,NaN,NaN,NaN,"산포,영국","김영사,왕립원예협회","식물학자,과학,일러스트,식물들,경이,투쟁사,담다,식물학자,노트,신혜우,그림,김영사,...","식물학자,식물들,신혜우,잔뿌리,dust,seed,개구리밥,종자식물,회화작품,신기록,...",식물학자의 노트\n신혜우 글 그림\n김영사 | 280쪽 | 1만9800원\n\n전 ...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
9,1.100101e+06,20210430,경향신문,배문규 기자 sobbell@kyunghyang.com,[책과 삶]옛 그림 지도로 떠나는 탐라 명소 역사 대장정,지역>제주,문화>미술_건축,문화>출판,NaN,NaN,NaN,"김남길,최열,이형상","제주목,산방굴,제주,성산일출봉,정방폭포,탐라","원도심,탐라왕국,제주,조선","그림,지도,탐라,명소,역사,대장정,그림,제주,최열,지음,혜화1117,

In [214]:
len(df)

17711

In [224]:
df = df[~df.title.str.contains('세계읽기')]

In [228]:
df[df.name.str.contains('플랫팀', na=False)]

,identical,date,press,name,title,c1,c2,c3,a1,a2,a3,person,place,institute,keyword,topkeyword,body,url,tf
223,1.100101e+06,20210429,경향신문,플랫팀 twitter.com/flatflat38,‘부동산 투기’는 왜 아내의 일이 되었나[플랫],사회>여성,NaN,NaN,NaN,NaN,NaN,"김의겸,중산,최,최시현","한국,수도권","아시아도시사회센터,청와대,열린민주당,연세대,대소사,서울대","부동산,투기,아내,플랫,남성,정치인,고위,관료,투기,아내,주장,김의겸,김의겸,청와대...","주택실천,한국,중산층,김의겸,대변인,최시현,flatflat38,그중,연세대,서울대,...",남성 정치인과 고위 관료들은 왜 투기 문제가 불거질 때마다 “아내가 한 일이며 나는...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
297,1.100101e+06,20210429,경향신문,플랫팀 twitter.com/flatflat38,“DO TOUCH” 생리대와 월경컵 경험 공유하는 ‘월경상점’[플랫],사회>여성,문화>문화일반,NaN,NaN,NaN,NaN,"안지혜,여자친구,유해성,김수천","서울 동작구,한국,오가면,강원,대방동,파동","인천,아시아나항공,청구","TOUCH,생리대,공유,월경,경험,월경상점,플랫,서울,동작구,대방동,위치,살림,여성...","생리대,월경컵,온라인,생리휴가,대방동,여자친구,사람들,월경용품,생리현상,flatfl...",지난 24일 서울 동작구 대방동에 위치한 여성 창업 공간 ‘스페이스 살림’ 건물 1...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
422,1.100101e+06,20210428,경향신문,플랫팀 twitter.com/flatflat38,생리휴가 신청한 승무원들에게 ‘생리 증명’ 요구한 항공사 대표 대법원도 “유죄”[플랫],사회>미디어,사회>노동_복지,사회>여성,NaN,NaN,NaN,"이동원,김 전,김수천",NaN,"대법원,아시아나항공,청구","생리,휴가,신청,승무원들,생리,증명,요구,항공사,대표,대법원,유죄,플랫,승무원들,신...","생리휴가,생리현상,대법원,승무원,보건휴가,근로기준법,부여,생리용품,1심,이동원,fl...",승무원들이 신청한 생리휴가를 거부한 혐의로 재판에 넘겨진 아시아나항공 전 대표에게 ...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
464,1.100101e+06,20210428,경향신문,플랫팀 twitter.com/flatflat38,"여성들의 이름을 불러주자, 우리 서로 찾을 수 있도록[플랫]",문화>출판,문화>방송_연예,문화>전시_공연,범죄>성범죄>성폭행,범죄>범죄일반>살인,NaN,정재은,호명,"진로,KBS","여성들,이름,플랫,할머니,산소,다과,남동생,농촌,국도,매점,마을,이름,상점,사이,정...","영화감독,사장님,모던코리아,여성들,정재은,연출자,에이드리엔,김부남,복길,flatfl...",할머니의 산소 앞에 놓을 다과를 사기 위해 남동생과 나는 구불구불한 농촌 국도를 달...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
568,1.100101e+06,20210427,경향신문,플랫팀 twitter.com/flatflat38,‘불구’가 혐오 아닌 자긍심의 말이 될 때[플랫],사회>장애인,문화>출판,NaN,사회>사회문제>노예,사회>사회갈등>시위,NaN,"버락 오바마,미셸 오바마,일라이","미국,유럽,차도,코다,북아메리카,워싱턴,동아시아,제네드,솔지,뉴욕,워싱턴시,샌프란시...","보건복지부,미국,딜란시,매디슨,넷플릭스,FBI,아이와,닐슨,나비여,제네드,영권,프릭...","혐오,자긍심,플랫,이길보라,논픽션,세계,전학,학년당,학급,시골,학교,동생,부모님,적...","장애인,제네드,미국,몸들,농인,부모님,재활법,토착민,전화선,비장애인,데이비드,청인,...",이길보라의 논픽션의 세계 \n\n\n\n4학년 1반으로 전학했다. 학년당 한 학급만...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16910,1.100101e+06,20201222,경향신문,플랫팀 twitter.com/flatflat38,“낙태죄 폐지 다음은 인권으로서 ‘재생산권’을 말해야 한다”[플랫],사회>여성,사회>노동_복지,NaN,범죄>기업범죄>거래제한,범죄>범죄일반>살인,사회>사회문제>노예,"김정혜,최현정,구하라,김선혜,이은진","카이로,일본,강남역,한국,혜화역,북경","셰어 연구원,보건복지부,독일,양성평등정책위원회,한국여성정책연구원,국회,한국,복지부,...","낙태죄,폐지,인권,재생산권,플랫,낙태죄,2021년,헌법재판소,4월,형법,조항,낙태죄...","기본법,재생산,재생산권,낙태죄,한국,김선혜,기본법안,모자보건법,사유리,난임,이은진,...",낙태죄 없는 2021년이 올까.\n\n\n\n헌법재판소는 지난해 4월11일 형법의 ...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
17092,1.100101e+06,20201221,경향신문,플랫팀 twitter.com/flatflat38,‘스토킹 처벌법’을 만드는 데 시간이 오래 걸리는 이유[플랫],사회>사건_사고,사회>여성,NaN,NaN,NaN,NaN,"황운하,송란희,승재현,남인순,임호선,서범수,정춘숙",구치소,"경찰서장,진로,승재현,민주당,검찰,청구,경찰,국회,한국여성의전화,형사정책연구원,서장...","스토킹,처벌법,플랫,법무부,입법예고,스토킹처벌법,수정,입법예고,2년,2년,입법예고안...","피해자,처벌법,스토킹처벌법,관계자,잠정조치,법무부,남인순,승재현,입법예고안,반의사불...",법무부가 2018년 5월에 입법예고한 스토킹처벌법이 일부 수정을 거쳐 지난 11월 ...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
17349,1.100101e+06,20201218,경향신문,플랫팀 twitter.com/flatflat38,"‘보상으로서의 여성’ 룸살롱 연출, 언제까지 봐야하나[플랫]",문화>방송_연예,문화>영화,문화>출판,범죄>성범죄>성폭행,범죄>성범죄>성매매,NaN,"신혜선,최진혁,한준혁,손병호,김소용","장봉환,서울,옥타곤,조선,한국,철인왕후","강남구,SBS,현대,옥타곤,버닝썬,JTBC,한국,트위터,tvN,서울 YWCA,방송소...","보상,여성,룸살롱,연출,플랫,옷고름,사람,남성,현대,한국,영혼,김소용,조선,시대,중...","룸살롱,유흥업소,옥타곤,철인왕후,김소용,나성원,허쉬,조선,문제의식,시청자,성폭행,서...",“오늘 내 옷고름을 풀 사람은 누구?”\n\n\n\n현대 한국 남성의 영혼이 깃든 ...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN
17397,1.100101e+06,20201218,경향신문,플랫팀 twitter.com/flatflat38,20년간 여성을 변론한 김수정 변호사가 말하는 ‘아주 오래된 유죄’[플랫],사회>여성,사회>교육_시험,사회>미디어,사회>사회문제>미성년범죄,NaN,NaN,김수정,"서초동,서울,미군 기지촌 위안부,호주","정부,헌법재판소,국회,헌재,한국,배드파더스,공청","20년,여성,변론,김수정,변호사,유죄,플랫,20년,법정,여성,변론,김수정,변호사,역...","변호사,피해자,미군,서울,임신중지,기지촌,사무실,여성들,김수정,집단감염,헌법재판소,...",지난 20년간 법정에서 여성을 변론해온 김수정 변호사(51)는 역설적이게도 “법정에...,http://news.khan.co.kr/kh_news/khan_art_view.h...,NaN


In [212]:
len(df)

18

In [70]:
text_556 = pd.read_excel('C:/Develops/Project_multi_A3/scraping/text/text_556+1.xls', header=None)

In [71]:
text_556 = text_556.values.tolist()

In [72]:
text_556[0]

['젊은 사자의 포효가 그치지 않고 있다. 삼성 토종 에이스 원태인(21)이 1위를 놓고 다투고 있는 LG전에서도 완벽한 투구를 선보이며 4월 가장 강력한 MVP 후보로 올라섰다.원태인은 30일 대구 삼성라이온즈파크에서 열린 LG와의 시즌 첫 경기에서 선발로 등판해 7이닝 동안 5안타 무실점으로 LG 타선을 완벽하게 틀어막았다. 이날 경기결과로 평균자책이 1.16까지 떨어진 원태인은 LG 앤드류 수아레즈(1.23)를 제치고 평균자책 리그 1위로 올라섰으며, 팀이 이날 4-0으로 승리해 다승 부문 역시 팀동료 데이비드 뷰캐넌, 이날 SSG전 승리투수가 된 두산 아리엘 미란다와 함께 4승으로 공동선두를 형성했다.원태인의 호투는 도쿄 올림픽을 3개월여 앞두고 선발진 구축에 고심하고 있는 김경문호에도 희소식이 될 것으로 보인다. 지난해 초반 NC 구창모가 그랬듯 원태인의 4월 투구내용은 국내 투수 중 가장 두드러진다. 원태인은 이날까지 올해 등판 5경기 가운데 4경기를 퀄리티스타트(6이닝 이상 3자책 이하) 피칭으로 장식하고 있다.1회 삼자범퇴로 상대를 돌려세운 원태인은 2회 이형종에게 2루타를 맞았지만 이천웅과 김민성을 삼진과 땅볼로 처리했다.3회를 다시 삼자범퇴로 처리한 원태인은 4회 위기를 맞았다. 오지환의 볼넷에 이은 김현수의 우전안타로 만든 무사 1·2루 위기에서 4번 로베르토 라모스를 삼진, 이형종을 뜬공, 이천웅을 땅볼로 정리하면서 위기에서 벗어났다.삼성은 3회말 2사 2·3루에서 나온 호세 피렐라의 땅볼로 1점을 선취하고 5회 다시 피렐라가 바뀐 투수 이정용의 투구를 좌측 담장 너머로 넘기면서 앞서갔다. 7회에는 밀어내기 추가점, 8회 이학주의 솔로포도 나왔다. 삼성은 LG를 0.5경기 차로 제치고 선두로 나섰다.잠실에서는 외국인 투수 아리엘 미란다가 6이닝 3안타 무실점으로 호투한 두산이 4회말 안타 7개와 사사구 2개를 묶어 대거 7득점하면서 SSG를 9-4으로 이겼다. 수원에서는 선발 고영표가 역시 7이닝 7안타 3실점으로 4경기 연속 퀄리티스타트

In [73]:
text_558_1043 = pd.read_excel('C:/Develops/Project_multi_A3/scraping/text/text_558_1043+1.xls', header=None)

In [74]:
text_558_1043 = text_558_1043.values.tolist()

In [75]:
text_1045_1325 = pd.read_excel('C:/Develops/Project_multi_A3/scraping/text/text_1045_1325+1.xls', header=None)

In [76]:
text_1045_1325 = text_1045_1325.values.tolist()

In [77]:
texts = text_556 + text_558_1043 + text_1045_1325

In [78]:
len(texts)

1313

In [79]:
len(text_556), len(text_558_1043), len(text_1045_1325)

(553, 482, 278)

In [25]:
text_1045_1325[0]

['-제한 없는 물류배송으로 B2B 택배서비스 부분 브랜드 충성도와 신뢰도, 품질에서 높은 평가경동물류 경동택배가 20일 서울 신라호텔에서 개최된 ‘2021 대한민국 대표 브랜드 대상’에서 택배부문 대한민국 대표브랜드로 선정됐다고 22일 밝혔다.올해 16회를 맞이한 2021년 대한민국 대표 브랜드 대상은 매년 각 분야에 지속적인 혁신을 통해 소비자들로부터 사랑받는 브랜드를 가려 대한민국을 대표하는 브랜드를 선정한다.올해 2월, 19일간 imbc, 동아닷컴, 한경닷컴 3사의 인터넷 회원과 대한민국 소비자를 대상으로 진행한 이번 조사를 통해 택배부문에서 브랜드 충성도와 신뢰도 품질 면에서 높은 평가를 받아 경동택배가 브랜드 경쟁력을 가진 대한민국 대표브랜드로 선정되어 수상하였다.경동물류의 경동택배는 중량화물 중 전국 1200여 개가 넘는 영업소를 기반으로 독보적인 배송 네트워크를 구축하여 언제든, 어디든, 무엇이든 부피나 무게에 제한 없이 보낼 수 있는 택배이다.경동택배 관계자는 앞으로도 소비자에게 차별적 가치를 제공하고 지속적인 브랜드 커뮤니케이션환경을 만들어 고객사와 소비자에게 더 나은 생활을 만드는 대한민국 대표 택배 브랜드로 물류 혁신에 최선을 다할 것을 다짐했다.']

In [208]:
i = 297
df.title[i]

'“DO TOUCH” 생리대와 월경컵 경험 공유하는 ‘월경상점’[플랫]'

In [209]:
texts[i]

['김수영 구청장이 일본후쿠시마 원전 오염수 해양방류 철회를 요구하는 ‘지속가능발전 지방정부협의회’의 성명서 발표에 동참했다고 서울 양천구가 29일 전했다.지속가능발전 지방정부협의회는 이날 성명서를 통해 “인간 생명과 생태계를 위협하는 후쿠시마 방사능 오염수 해양방류를 철회하고 대안을 마련해야하며, 모든 폐기물의 처리과정을 국제사회와 투명하게 협의하라”고 요구했다. 또 일본정부의 방사능 오염수 해양방류 결정이 철회될 수 있도록 국제법 제소 등 정부의 강력한 대응도 요구했다.김 구청장은 “일본 후쿠시마 원전 오염수 해양방류는 주변 몇몇 국가에만 영향을 미치는 것이 아니라 생태계 전체를 위협하는 행위로 국제적 차원의 대응이 필요하다”며 “이는 국제사회가 합의한 공동목표이자 사회적 가치”라고 밝혔다. 이어 “협의회 회원 지방정부들과 함께 후쿠시마 원전 오염수 해양방출 철회를 위해 적극 대응할 것”이라고 강조했다.지속가능발전 지방정부협의회는 지방정부 간 협력과 중앙정부와의 유기적 연계를 통한 전략 마련을 위해 전국의 29개 기초단체가 참여하는 협의체다.']

In [ ]:
44 [김진호의 세계읽기]‘푸틴 정적’ 감싸는 바이든의 미국, 속내는
223 '‘부동산 투기’는 왜 아내의 일이 되었나[플랫]'

In [180]:
texts.insert(223, 'None')